In [ ]:

using Markdown
using InteractiveUtils
using Graphs, Random, Statistics
using Plots, Colors
using ITensors, ITensorMPS, LinearAlgebra
using JLD2 
using ITensors: entropy 





In [2]:

function create_MPS(L::Int)
    sites = siteinds("S=1/2", L; conserve_qns=true)
    initial_state = [isodd(i) ? "Up" : "Dn" for i in 1:L]
    ψ₀ = randomMPS(sites, initial_state)
    return ψ₀, sites
end



create_MPS (generic function with 1 method)

In [3]:
"""
Creates a weighted adjacency matrix for a completely connected graph.
"""
function create_weighted_adj_mat(N::Int, σ::Float64; μ::Float64=1.0)
    if σ == 0.0
        A = ones(Float64, N, N)
        A -= Matrix{Float64}(I, N, N)
        return A
    end
    A = zeros(Float64, N, N)
    for i in 1:N, j in (i+1):N
        weight = μ + σ * randn()
        A[i, j] = A[j, i] = weight
    end
    return A
end

create_weighted_adj_mat

In [4]:
"""
Creates the MPO for the XXZ Hamiltonian on a graph with weighted interactions.
"""
function create_weighted_xxz_mpo(N::Int, adj_mat, sites; J::Float64, Δ::Float64)
    ampo = OpSum()
    for i in 1:N-1
        for j in i+1:N
            coupling_strength = adj_mat[i, j]
            if coupling_strength != 0.0
                ampo += coupling_strength * (J / 2), "S+", i, "S-", j
                ampo += coupling_strength * (J / 2), "S-", i, "S+", j
                ampo += coupling_strength * (J * Δ), "Sz", i, "Sz", j
            end
        end
    end
    return MPO(ampo, sites)
end

create_weighted_xxz_mpo

In [ ]:

# Calculates Von Neumann entropy for the central bipartition
# Runs for specified sigma values and averages over graphs if sigma != 0
function run_entropy_simulation()

    N_range = 10:2:100 # System sizes from 10 to 100 [cite: 519]
    sigma_values_to_run = [0.0, 0.002] # Sigma values for Fig 12 & 13 [cite: 519, 552]
    num_graphs_avg = 10 # Number of graphs to average over for sigma != 0 

    # DMRG Parameters (consistent with previous code)
    num_sweeps = 30
    max_bond_dim_limit = 250
    cutoff = 1E-10
    μ = 1.0
    J_coupling = -0.5 # As used in previous MPO creation
    Delta_coupling = 0.5 # As used in previous MPO creation

    # Dictionary to store results: sigma => vector of entropies for each N
    results = Dict{Float64, Vector{Float64}}()

    println("Starting entropy simulations...")

    for σ in sigma_values_to_run
        println("Running for σ = $σ")
        entropies_for_N = Float64[] # Store results for this sigma

        for N in N_range
            # Ensure N is even for a clear central bipartition N/2
            # Although ITensors handles indices correctly, the report implies N/2 split.
            # If odd N is needed, adjust 'b' calculation.
            if N % 2 != 0
                println("Skipping N=$N as it's odd.")
                continue
            end

            b = N ÷ 2 # Central bond index for bipartition [N/2 | N/2+1] 
            entropies_for_avg = Float64[] # Store entropies for averaging

            num_runs = (σ == 0.0) ? 1 : num_graphs_avg # Only 1 run needed for sigma=0

            print("  N = $N: ")
            for run in 1:num_runs
                if σ != 0.0 print("$run ") end

                ψ₀, sites = create_MPS(N)
                adj_mat = create_weighted_adj_mat(N, σ; μ=μ)
                H_mpo = create_weighted_xxz_mpo(N, adj_mat, sites; J=J_coupling, Δ=Delta_coupling)

                sweeps = Sweeps(num_sweeps)
                setmaxdim!(sweeps, max_bond_dim_limit)
                setcutoff!(sweeps, cutoff)

                # Use noise term for better convergence, especially with randomness
                # Small noise added initially and decreased over sweeps
                noise_vals = LinRange(1E-6, 1E-10, num_sweeps)
                setnoise!(sweeps, noise_vals...)

                # Run DMRG (outputlevel=0 suppresses sweep output)
                _, ψ_gs = dmrg(H_mpo, ψ₀, sweeps; outputlevel=0)

                # Calculate Von Neumann entropy across the central bond 'b' 
                S = entropy(ψ_gs, b)
                push!(entropies_for_avg, S)
            end
            println() # Newline after runs for a given N

            avg_entropy = mean(entropies_for_avg)
            push!(entropies_for_N, avg_entropy)
            println("    Avg Entropy = $avg_entropy")

        end # End loop N

        results[σ] = entropies_for_N

    end # End loop sigma

    println("Simulations finished.")
    # Filter N_range to only include even numbers used
    actual_N_range = filter(isodd, N_range) ? N_range : filter(iseven, N_range) # Crude filter, adjust if N_range has odds
    if isempty(actual_N_range) actual_N_range = N_range end # Fallback if N_range was all odd

    return results, actual_N_range
end

run_simulation_and_plot_delta (generic function with 1 method)

In [ ]:


function entropy_plot(N_range, results; backend=gr)
    backend() # Set plotting backend (e.g., gr(), plotlyjs())
    N_values = collect(N_range)

    plt = plot(title="Average Von Neumann Entropy of the Central Bipartition",
               xlabel="Number of Nodes",
               ylabel="Average Entropy",
               legend=:topleft,
               grid=true,
               minorgrid=true,
               gridalpha=0.2,
               minorgridalpha=0.1)

    # Define colors to potentially match report (adjust as needed)
    colors = Dict(0.0 => :purple, 0.002 => :darkorange) # Or use Plots default

    sigmas_sorted = sort(collect(keys(results)))

    for σ in sigmas_sorted
        label_str = "σ=$σ"
        color = get(colors, σ, :auto) # Use defined color or auto
        entropy_values = results[σ]

        # Ensure entropy_values match N_values length if odd Ns were skipped
         if length(entropy_values) != length(N_values)
            println("Warning: Mismatch in lengths for σ=$σ. N_values: $(length(N_values)), Entropies: $(length(entropy_values))")
            # Attempt to plot anyway, might error or look wrong if lengths mismatch significantly
        end

        # Plot points and lines
        scatter!(plt, N_values, entropy_values, label=label_str, color=color, markersize=4, markerstrokewidth=0.5, markerstrokecolor=:black)
        plot!(plt, N_values, entropy_values, label="", color=color, linewidth=1.5) # Line connecting points
    end

    return plt
end

In [ ]:


function entropy_plot_log(N_range, results; backend=gr)
    backend() # Set plotting backend
    N_values_full = collect(N_range)
    # Ensure we only take log of positive values, filter if needed (should be fine for N>=10)
    N_log_values = log2.(N_values_full)

    plt = plot(title="Average Von Neumann Entropy of the Central Bipartition",
               xlabel="Number of Nodes (log2 scale)",
               ylabel="Average Entropy",
               legend=:topleft,
               grid=true,
               minorgrid=true,
               gridalpha=0.2,
               minorgridalpha=0.1)

    colors = Dict(0.0 => :purple, 0.002 => :darkorange)
    sigmas_sorted = sort(collect(keys(results)))

    for σ in sigmas_sorted
        label_str = "σ=$σ"
        color = get(colors, σ, :auto)
        entropy_values = results[σ]

         if length(entropy_values) != length(N_log_values)
            println("Warning: Mismatch in lengths for σ=$σ. N_log_values: $(length(N_log_values)), Entropies: $(length(entropy_values))")
        end

        # Plot points and lines using log2(N) axis
        scatter!(plt, N_log_values, entropy_values, label=label_str, color=color, markersize=4, markerstrokewidth=0.5, markerstrokecolor=:black)
        plot!(plt, N_log_values, entropy_values, label="", color=color, linewidth=1.5)
    end

     # Optional: Customize x-ticks to show 2^{x} format if desired
     # This requires more complex tick formatting, might be simpler to leave as log values
     xticks_vals = [log2(v) for v in [4, 8, 16, 32, 64]] # Example ticks
     xtick_labels = ["2^{$(Int(log2(v)))}" for v in [4, 8, 16, 32, 64]]
     # Filter ticks to be within the actual data range
     valid_ticks = [(t, l) for (t, l) in zip(xticks_vals, xtick_labels) if minimum(N_log_values) <= t <= maximum(N_log_values)]
     if !isempty(valid_ticks)
         plot!(plt, xticks=(first.(valid_ticks), last.(valid_ticks)))
     end


    return plt
end

WebIO._IJuliaInit()

┌ Warning: backend `PlotlyJS` is not installed.
└ @ Plots C:\Users\Ethan\.julia\packages\Plots\8ZnR3\src\backends.jl:45


Data saved successfully.



In [ ]:

entropy_results, N_range_used = run_entropy_simulation();

ent_plot = entropy_plot(N_range_used, entropy_results);
display(ent_plot)

ent_plot_log = entropy_plot_log(N_range_used, entropy_results);
display(ent_plot_log)

using JLD2
filename_entropy = "entropy_vs_N_data.jld2"
jldsave(filename_entropy; entropy_results, N_range_used)


load_data_and_plot (generic function with 1 method)